In [2]:
# extract "corpus"
%reset -f
import pandas as pd
print('reading...')
df = pd.read_csv('./train_merged.csv')
print('preprocessing...')
df['creative_id'] = df['creative_id'].astype(str)
df['advertiser_id'] = df['advertiser_id'].astype(str)
df['ad_id'] = df['ad_id'].astype(str)
print('extracting...')
# need to be str type for w2v
creative_id = df.groupby('user_id')['creative_id'].apply(list)
advertiser_id = df.groupby('user_id')['advertiser_id'].apply(list)
ad_id = df.groupby('user_id')['ad_id'].apply(list)
del df
print('reading...')
df = pd.read_csv('./predict_merged.csv')
print('preprocessing...')
df['creative_id'] = df['creative_id'].astype(str)
df['advertiser_id'] = df['advertiser_id'].astype(str)
df['ad_id'] = df['ad_id'].astype(str)
print('extracting...')
creative_id = pd.concat([creative_id, df.groupby('user_id')['creative_id'].apply(list)])
advertiser_id = pd.concat([advertiser_id, df.groupby('user_id')['advertiser_id'].apply(list)])
ad_id = pd.concat([ad_id, df.groupby('user_id')['ad_id'].apply(list)])
del df
display(creative_id)

reading...
preprocessing...
extracting...
reading...
preprocessing...
extracting...


user_id
1          [71691, 90171, 122032, 209778, 821396, 877468,...
2          [13069, 15558, 22013, 39714, 63441, 96192, 155...
3          [66009, 72533, 392052, 593522, 599128, 661347,...
4          [31070, 39588, 72989, 215041, 574787, 589886, ...
5          [24333, 43235, 75011, 296145, 350759, 795508, ...
                                 ...                        
3999996    [12916, 30856, 54100, 65314, 116356, 167175, 1...
3999997    [14993, 50445, 66299, 271727, 390953, 641757, ...
3999998    [24188, 54150, 100456, 316582, 464055, 771243,...
3999999    [840, 9148, 71770, 72039, 78406, 95215, 115084...
4000000    [1973, 4160, 48855, 49574, 58631, 66621, 71689...
Name: creative_id, Length: 1900000, dtype: object

In [4]:
creative_id.to_numpy()[0]

['71691',
 '90171',
 '122032',
 '209778',
 '821396',
 '877468',
 '1683713',
 '1940159',
 '2087846',
 '2456345',
 '2456345',
 '3068256',
 '3592928']

In [3]:
# save to file
import pickle

with open('./creative_id.txt', 'wb') as f:
    pickle.dump(creative_id.to_numpy().tolist(), f)
with open('./advertiser_id.txt', 'wb') as f:
    pickle.dump(advertiser_id.to_numpy().tolist(), f)
with open('./ad_id.txt', 'wb') as f:
    pickle.dump(ad_id.to_numpy().tolist(), f)

In [18]:
# w2v
%reset -f

# !pip install gensim
# !pip install Cython
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.models
from gensim.utils import RULE_DISCARD, RULE_KEEP

In [19]:
# creative_id
import pickle
print('loading...')
with open('./creative_id.txt', 'rb') as f:
    corpus = pickle.load(f)
print('training...')
model = gensim.models.Word2Vec(sentences=corpus, workers=12, size=300, min_count=2,
                               hs=0, window=100, sg=1, 
                              trim_rule=lambda word, count, min_count: RULE_DISCARD if word=='0' else RULE_KEEP)
# model.build_vocab(sentences=corpus, update=True,
#                  trim_rule=lambda word, count, min_count: RULE_DISCARD if word=='0' else RULE_KEEP,
#                  progress_per=500000)
# model.train(sentences=corpus, total_examples=model.corpus_count, epochs=model.iter)

loading...


2020-06-05 03:55:59,635 : INFO : collecting all words and their counts
2020-06-05 03:55:59,636 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-05 03:55:59,818 : INFO : PROGRESS: at sentence #10000, processed 335578 words, keeping 150228 word types


training...


2020-06-05 03:55:59,973 : INFO : PROGRESS: at sentence #20000, processed 673775 words, keeping 252881 word types
2020-06-05 03:56:00,135 : INFO : PROGRESS: at sentence #30000, processed 1012932 words, keeping 337989 word types
2020-06-05 03:56:00,334 : INFO : PROGRESS: at sentence #40000, processed 1344506 words, keeping 411658 word types
2020-06-05 03:56:00,533 : INFO : PROGRESS: at sentence #50000, processed 1683247 words, keeping 480886 word types
2020-06-05 03:56:00,717 : INFO : PROGRESS: at sentence #60000, processed 2018755 words, keeping 542999 word types
2020-06-05 03:56:00,882 : INFO : PROGRESS: at sentence #70000, processed 2356409 words, keeping 601314 word types
2020-06-05 03:56:01,080 : INFO : PROGRESS: at sentence #80000, processed 2694811 words, keeping 655662 word types
2020-06-05 03:56:01,266 : INFO : PROGRESS: at sentence #90000, processed 3029095 words, keeping 706318 word types
2020-06-05 03:56:01,434 : INFO : PROGRESS: at sentence #100000, processed 3367959 words, 

In [20]:
model.save('./w2v_creative_id')

2020-06-05 06:08:03,011 : INFO : saving Word2Vec object under ./w2v_creative_id, separately None
2020-06-05 06:08:03,012 : INFO : storing np array 'vectors' to ./w2v_creative_id.wv.vectors.npy
2020-06-05 06:08:09,706 : INFO : not storing attribute vectors_norm
2020-06-05 06:08:09,708 : INFO : storing np array 'syn1neg' to ./w2v_creative_id.trainables.syn1neg.npy
2020-06-05 06:08:39,716 : INFO : not storing attribute cum_table
2020-06-05 06:08:51,961 : INFO : saved ./w2v_creative_id


In [1]:
import gensim.models
wv = gensim.models.Word2Vec.load('./w2v_creative_id').wv

In [6]:
wv.vocab

{'71691': <gensim.models.keyedvectors.Vocab at 0x7f0dce0f8668>,
 '90171': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fa20>,
 '122032': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fa90>,
 '209778': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fb00>,
 '821396': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fb70>,
 '877468': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fbe0>,
 '1683713': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fc50>,
 '1940159': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fcc0>,
 '2087846': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fd30>,
 '2456345': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fda0>,
 '3068256': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fe10>,
 '3592928': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fe80>,
 '13069': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05fef0>,
 '15558': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05ff60>,
 '22013': <gensim.models.keyedvectors.Vocab at 0x7f0dcd05ffd0>,
 '39714': <gensim.models

In [10]:
wv[['71691', '90171']]

array([[ 6.99492469e-02, -8.79556090e-02,  1.07327685e-01,
        -1.49118572e-01, -2.01712251e-01, -3.56417418e-01,
        -6.90838993e-01,  6.56082779e-02,  2.55345970e-01,
        -1.85727239e-01,  1.23333365e-01, -3.42110425e-01,
        -3.84708673e-01,  1.97496176e-01,  1.90465659e-01,
        -7.92665333e-02,  1.05615512e-01,  5.69597483e-02,
        -3.68587732e-01,  2.25398302e-01,  2.31962651e-01,
        -2.72898316e-01, -1.79060370e-01,  2.80694902e-01,
         2.77880728e-01, -3.18279654e-01, -2.64256835e-01,
         5.90207689e-02,  9.36662629e-02, -2.43923709e-01,
        -1.30285695e-01, -4.56965230e-02,  5.26741445e-02,
        -1.11203671e-01, -3.01120654e-02,  2.46843308e-01,
        -1.97585389e-01, -1.03612177e-01, -2.32790321e-01,
        -3.33480209e-01, -2.24022880e-01, -3.16946238e-01,
         2.00075760e-01, -3.59847583e-02,  3.02354932e-01,
         1.36064172e-01,  7.62494430e-02, -1.27569303e-01,
        -6.20420612e-02,  4.38575298e-02, -5.45216799e-0

In [2]:
# creative_id, gender
%reset -f
# !pip install gensim

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.models
import pandas as pd
import numpy as np
wv = gensim.models.Word2Vec.load('./w2v_creative_id').wv

2020-06-15 12:21:15,725 : INFO : loading Word2Vec object from ./w2v_creative_id
2020-06-15 12:21:28,150 : INFO : loading wv recursively from ./w2v_creative_id.wv.* with mmap=None
2020-06-15 12:21:28,151 : INFO : loading vectors from ./w2v_creative_id.wv.vectors.npy with mmap=None
2020-06-15 12:22:01,056 : INFO : setting ignored attribute vectors_norm to None
2020-06-15 12:22:01,058 : INFO : loading vocabulary recursively from ./w2v_creative_id.vocabulary.* with mmap=None
2020-06-15 12:22:01,059 : INFO : loading trainables recursively from ./w2v_creative_id.trainables.* with mmap=None
2020-06-15 12:22:01,059 : INFO : loading syn1neg from ./w2v_creative_id.trainables.syn1neg.npy with mmap=None
2020-06-15 12:22:33,785 : INFO : setting ignored attribute cum_table to None
2020-06-15 12:22:33,786 : INFO : loaded ./w2v_creative_id


In [3]:
print('reading...')
df = pd.read_csv('./train_merged.csv')
print('preprocessing...')
df['creative_id'] = df['creative_id'].astype(str)
print('transforming...')
creative_id = df.groupby('user_id')['creative_id'].apply(list)
creative_id

reading...
preprocessing...
transforming...


user_id
1         [71691, 90171, 122032, 209778, 821396, 877468,...
2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
3         [66009, 72533, 392052, 593522, 599128, 661347,...
4         [31070, 39588, 72989, 215041, 574787, 589886, ...
5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                ...                        
899996    [92193, 114074, 654526, 931537, 939409, 145880...
899997    [7400, 24333, 30210, 60330, 69204, 103918, 103...
899998    [71752, 648402, 1251649, 1370659, 1779124, 230...
899999    [12838, 12838, 12838, 12838, 122834, 279342, 3...
900000    [234063, 285502, 423617, 1224107, 1405713, 180...
Name: creative_id, Length: 900000, dtype: object

In [4]:
import tensorflow as tf
frac = 0.6
train_x = creative_id[:int(frac*len(creative_id))]
valid_x = creative_id[int(frac*len(creative_id)):]
user_info = pd.read_csv('./user_info.csv')
gender = user_info['gender']
train_y = gender[:int(frac*len(creative_id))]
valid_y = gender[int(frac*len(creative_id)):]
train_x, train_y

(user_id
 1         [71691, 90171, 122032, 209778, 821396, 877468,...
 2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
 3         [66009, 72533, 392052, 593522, 599128, 661347,...
 4         [31070, 39588, 72989, 215041, 574787, 589886, ...
 5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                 ...                        
 539996    [11053, 42122, 130554, 134765, 389299, 418302,...
 539997    [1837001, 1845588, 2047669, 2609939, 2609940, ...
 539998    [14471, 70105, 670039, 672906, 1862575, 244271...
 539999    [29259, 53172, 96192, 421454, 425920, 1024894,...
 540000    [63444, 1399016, 1399016, 1399016, 1399016, 13...
 Name: creative_id, Length: 540000, dtype: object,
 0         0
 1         0
 2         1
 3         0
 4         0
          ..
 539995    1
 539996    0
 539997    0
 539998    0
 539999    1
 Name: gender, Length: 540000, dtype: int64)

In [5]:
def generate_w2v_train():
    for arr, gender in zip(train_x.to_numpy(), train_y):
        yield (wv[arr], gender)
        
train_dataset = tf.data.Dataset.from_generator(
    generate_w2v_train, (tf.float32, tf.int64), 
    (tf.TensorShape([None, 300]), tf.TensorShape([])))

In [6]:
iterator = train_dataset.make_one_shot_iterator()
iterator.get_next()

2020-06-15 12:25:53,271 : WARNING : From <ipython-input-6-43703b2e64f7>:1: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


(<tf.Tensor: id=31, shape=(13, 300), dtype=float32, numpy=
 array([[ 0.06994925, -0.08795561,  0.10732768, ..., -0.42854732,
         -0.24935117,  0.0914242 ],
        [ 0.13329802, -0.18502815,  0.21870264, ..., -0.37015957,
         -0.14289252, -0.07735917],
        [ 0.14588733, -0.11126285, -0.14611486, ...,  0.15282993,
         -0.11073759, -0.06555343],
        ...,
        [-0.1202803 ,  0.11371075,  0.38716465, ..., -0.06453274,
         -0.29583782,  0.17139201],
        [-0.18872088,  0.15168606,  0.18531603, ..., -0.01119228,
         -0.14554526,  0.20421647],
        [ 0.16552609,  0.4055918 ,  0.264571  , ..., -0.28462148,
         -0.42815202,  0.15195782]], dtype=float32)>,
 <tf.Tensor: id=32, shape=(), dtype=int64, numpy=0>)

In [7]:
dataset = train_dataset.padded_batch(128, (tf.TensorShape([None, 300]), tf.TensorShape([])))
iterator = dataset.make_one_shot_iterator()
iterator.get_next()

(<tf.Tensor: id=45, shape=(128, 175, 300), dtype=float32, numpy=
 array([[[ 0.06994925, -0.08795561,  0.10732768, ..., -0.42854732,
          -0.24935117,  0.0914242 ],
         [ 0.13329802, -0.18502815,  0.21870264, ..., -0.37015957,
          -0.14289252, -0.07735917],
         [ 0.14588733, -0.11126285, -0.14611486, ...,  0.15282993,
          -0.11073759, -0.06555343],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.11492804, -0.06971309,  0.04431322, ..., -0.2264956 ,
           0.19723506, -0.15028167],
         [-0.01100962, -0.10677161, -0.12601174, ..., -0.085996  ,
           0.1692722 ,  0.15154783],
         [-0.02378013,  0.10593007, -0.08805973, ..., -0.07255152,
           0.226015

In [8]:
tf.random.set_seed(13)
lstm_model = tf.keras.Sequential()
# lstm_model.add(tf.keras.layers.Input(shape=(None, 300))
lstm_model.add(tf.keras.layers.Masking(mask_value=0.0, input_shape=(None, 300)))
lstm_model.add(tf.keras.layers.LSTM(32))
lstm_model.add(tf.keras.layers.Dropout(0.5))
# lstm_model.add(tf.keras.layers.LSTM(16))
lstm_model.add(tf.keras.layers.Dense(16, activation='relu'))
lstm_model.add(tf.keras.layers.Dropout(0.5))
lstm_model.add(tf.keras.layers.Dense(1, activation='relu'))
lstm_model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=['accuracy'])
lstm_model.fit(x=dataset, epochs=3, verbose=1, workers=8, use_multiprocessing=True)

Epoch 1/3
4219/4219 [==============================] - 1160s 275ms/step - loss: 0.5669 - accuracy: 0.9149
Epoch 2/3
4219/4219 [==============================] - 1115s 264ms/step - loss: 0.5529 - accuracy: 0.9287
Epoch 3/3
4219/4219 [==============================] - 1113s 264ms/step - loss: 0.5505 - accuracy: 0.9299


In [9]:
!mkdir ./models/lstm_gender_dropout
with open('./models/lstm_gender_dropout/json', 'w') as f:
    f.write(lstm_model.to_json())
lstm_model.save_weights('./models/lstm_gender_dropout/weights.h5')

In [34]:
# lstm_model.save('./models/lstm_gender')

2020-06-10 12:02:47,375 : INFO : Assets written to: ./models/lstm_gender/assets


In [13]:
import tensorflow as tf
with open('./models/lstm_gender_dropout/json', 'r') as jsonfile:
    model = tf.keras.models.model_from_json(jsonfile.read())
model.load_weights('./models/lstm_gender_dropout/weights.h5')
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=['accuracy'])

In [14]:
def generate_w2v_valid():
    for arr, gender in zip(valid_x.to_numpy(), valid_y):
        yield (wv[arr], gender)
        
valid_dataset = tf.data.Dataset.from_generator(
    generate_w2v_valid, (tf.float32, tf.int64), 
    (tf.TensorShape([None, 300]), tf.TensorShape([])))

In [15]:
batch_dataset = valid_dataset.padded_batch(128, (tf.TensorShape([None, 300]), tf.TensorShape([])))
iterator = batch_dataset.make_one_shot_iterator()
iterator.get_next()

(<tf.Tensor: id=47958, shape=(128, 315, 300), dtype=float32, numpy=
 array([[[ 5.82091287e-02,  5.36561683e-02, -7.11442158e-02, ...,
          -1.48167804e-01, -7.47779831e-02, -6.68137819e-02],
         [ 1.37719154e-01,  9.29880142e-03,  3.03852201e-01, ...,
          -4.43445593e-01, -1.53363526e-01,  2.33793557e-02],
         [ 1.03078827e-01, -1.11570366e-01,  2.09671929e-02, ...,
           3.48292083e-01, -1.00248620e-01, -3.08283508e-01],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[ 4.26829457e-01, -2.82071203e-01, -1.69625312e-01, ...,
          -2.53072828e-01, -4.25310820e-01, -1.21070750e-01],
         [ 4.268294

In [16]:
valid_loss, valid_accuracy = model.evaluate(x=batch_dataset, workers=8,
                                                 use_multiprocessing=True)
print('validation loss', valid_loss, 'validation accuracy', valid_accuracy)

2813/2813 [==============================] - 400s 142ms/step - loss: 0.5445 - accuracy: 0.9354
validation loss 0.5445157364619995 validation accuracy 0.935425


In [17]:
import pandas as pd
import numpy as np
print('reading...')
predict_merged = pd.read_csv('./predict_merged.csv')
print('preprocessing...')
predict_merged['creative_id'] = predict_merged['creative_id'].astype(str)
print('transforming...')
predict_creative_id = predict_merged.groupby(by=['user_id'])['creative_id'].apply(list)
predict_creative_id

reading...
preprocessing...
transforming...


user_id
3000001    [103064, 351878, 593698, 665090, 1508864, 1711...
3000002    [12838, 12838, 37295, 37298, 37298, 37298, 475...
3000003    [20503, 36634, 41212, 50485, 68601, 73137, 115...
3000004    [9870, 63715, 63715, 63715, 94025, 108656, 157...
3000005    [17535, 26793, 40625, 49682, 71219, 123860, 16...
                                 ...                        
3999996    [12916, 30856, 54100, 65314, 116356, 167175, 1...
3999997    [14993, 50445, 66299, 271727, 390953, 641757, ...
3999998    [24188, 54150, 100456, 316582, 464055, 771243,...
3999999    [840, 9148, 71770, 72039, 78406, 95215, 115084...
4000000    [1973, 4160, 48855, 49574, 58631, 66621, 71689...
Name: creative_id, Length: 1000000, dtype: object

In [18]:
def generate_w2v_predict():
    for arr in predict_creative_id.to_numpy():
        yield wv[arr]

predict_dataset = tf.data.Dataset.from_generator(
    generate_w2v_predict, tf.float32, tf.TensorShape([None, 300]))

In [19]:
predict_dataset = predict_dataset.padded_batch(32, tf.TensorShape([None, 300]))

iterator = predict_dataset.make_one_shot_iterator()
iterator.get_next()

<tf.Tensor: id=56726, shape=(32, 116, 300), dtype=float32, numpy=
array([[[ 0.14996356, -0.06210323, -0.19533257, ..., -0.05973113,
         -0.03714672, -0.14035879],
        [ 0.14884233, -0.46242303, -0.21204095, ..., -0.4200352 ,
         -0.3980428 , -0.03092831],
        [-0.1423606 ,  0.2580062 ,  0.34116986, ..., -0.08857717,
          0.1622483 ,  0.02941442],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.10590717, -0.661376  , -0.00094841, ..., -0.14658019,
         -0.14109349, -0.19529805],
        [ 0.10590717, -0.661376  , -0.00094841, ..., -0.14658019,
         -0.14109349, -0.19529805],
        [ 0.44052657, -0.22396596, -0.18939766, ..., -0.07724933,
          0.14604683, -0.17657009],
 

In [20]:
predict_gender = model.predict(x=predict_dataset, workers=8,
                                   use_multiprocessing=True, verbose=1)


31250/31250 [==============================] - 1456s 47ms/step


In [21]:
predict_gender

array([[0.       ],
       [4.543888 ],
       [2.5110087],
       ...,
       [0.       ],
       [0.       ],
       [0.       ]], dtype=float32)

In [22]:
submission = pd.DataFrame({'user_id': predict_creative_id.index})
submission['predicted_age'] = [0 for _ in range(1000000)]
submission['predicted_gender'] = (predict_gender > 0.5) + 1

submission

,user_id,predicted_age,predicted_gender
0,3000001,0,1
1,3000002,0,2
2,3000003,0,2
3,3000004,0,1
4,3000005,0,1
...,...,...,...
999995,3999996,0,1
999996,3999997,0,1
999997,3999998,0,1
999998,3999999,0,1


In [23]:
submission.to_csv('./submission_gender.csv', index=False)

In [5]:
# creative_id, age
# %reset -f
# !pip install gensim

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.models
import pandas as pd
import numpy as np
wv = gensim.models.Word2Vec.load('./w2v_creative_id').wv

2020-06-12 09:38:43,743 : INFO : loading Word2Vec object from ./w2v_creative_id
2020-06-12 09:39:02,773 : INFO : loading wv recursively from ./w2v_creative_id.wv.* with mmap=None
2020-06-12 09:39:02,774 : INFO : loading vectors from ./w2v_creative_id.wv.vectors.npy with mmap=None
2020-06-12 09:39:33,646 : INFO : setting ignored attribute vectors_norm to None
2020-06-12 09:39:33,647 : INFO : loading vocabulary recursively from ./w2v_creative_id.vocabulary.* with mmap=None
2020-06-12 09:39:33,647 : INFO : loading trainables recursively from ./w2v_creative_id.trainables.* with mmap=None
2020-06-12 09:39:33,648 : INFO : loading syn1neg from ./w2v_creative_id.trainables.syn1neg.npy with mmap=None
2020-06-12 09:40:04,606 : INFO : setting ignored attribute cum_table to None
2020-06-12 09:40:04,607 : INFO : loaded ./w2v_creative_id


In [4]:
print('reading...')
df = pd.read_csv('./train_merged.csv')
print('preprocessing...')
df['creative_id'] = df['creative_id'].astype(str)
print('transforming...')
creative_id = df.groupby('user_id')['creative_id'].apply(list)
creative_id

reading...
preprocessing...
transforming...


user_id
1         [71691, 90171, 122032, 209778, 821396, 877468,...
2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
3         [66009, 72533, 392052, 593522, 599128, 661347,...
4         [31070, 39588, 72989, 215041, 574787, 589886, ...
5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                ...                        
899996    [92193, 114074, 654526, 931537, 939409, 145880...
899997    [7400, 24333, 30210, 60330, 69204, 103918, 103...
899998    [71752, 648402, 1251649, 1370659, 1779124, 230...
899999    [12838, 12838, 12838, 12838, 122834, 279342, 3...
900000    [234063, 285502, 423617, 1224107, 1405713, 180...
Name: creative_id, Length: 900000, dtype: object

In [13]:
import tensorflow as tf
from tensorflow.keras.backend import manual_variable_initialization
manual_variable_initialization(True)
frac = 0.6
train_x = creative_id[:int(frac*len(creative_id))]
valid_x = creative_id[int(frac*len(creative_id)):]
user_info = pd.read_csv('./user_info.csv')
age = user_info['age'] - 1
train_y = age[:int(frac*len(creative_id))]
valid_y = age[int(frac*len(creative_id)):]
train_x, train_y

(user_id
 1         [71691, 90171, 122032, 209778, 821396, 877468,...
 2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
 3         [66009, 72533, 392052, 593522, 599128, 661347,...
 4         [31070, 39588, 72989, 215041, 574787, 589886, ...
 5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                 ...                        
 539996    [11053, 42122, 130554, 134765, 389299, 418302,...
 539997    [1837001, 1845588, 2047669, 2609939, 2609940, ...
 539998    [14471, 70105, 670039, 672906, 1862575, 244271...
 539999    [29259, 53172, 96192, 421454, 425920, 1024894,...
 540000    [63444, 1399016, 1399016, 1399016, 1399016, 13...
 Name: creative_id, Length: 540000, dtype: object,
 0         3
 1         9
 2         6
 3         4
 4         3
          ..
 539995    1
 539996    4
 539997    4
 539998    2
 539999    2
 Name: age, Length: 540000, dtype: int64)

In [14]:
def generate_train_age():
    for arr, age in zip(train_x.to_numpy(), train_y):
        yield (wv[arr], age)
        
train_age_dataset = tf.data.Dataset.from_generator(
    generate_train_age, (tf.float32, tf.int64),
    (tf.TensorShape([None, 300]), tf.TensorShape([])))

In [15]:
iterator = train_age_dataset.make_one_shot_iterator()
iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


2020-06-12 06:37:02,758 : WARNING : From <ipython-input-15-3f1bdb68ff88>:1: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


(<tf.Tensor: id=69284, shape=(13, 300), dtype=float32, numpy=
 array([[ 0.06994925, -0.08795561,  0.10732768, ..., -0.42854732,
         -0.24935117,  0.0914242 ],
        [ 0.13329802, -0.18502815,  0.21870264, ..., -0.37015957,
         -0.14289252, -0.07735917],
        [ 0.14588733, -0.11126285, -0.14611486, ...,  0.15282993,
         -0.11073759, -0.06555343],
        ...,
        [-0.1202803 ,  0.11371075,  0.38716465, ..., -0.06453274,
         -0.29583782,  0.17139201],
        [-0.18872088,  0.15168606,  0.18531603, ..., -0.01119228,
         -0.14554526,  0.20421647],
        [ 0.16552609,  0.4055918 ,  0.264571  , ..., -0.28462148,
         -0.42815202,  0.15195782]], dtype=float32)>,
 <tf.Tensor: id=69285, shape=(), dtype=int64, numpy=3>)

In [16]:
padded_age_dataset = train_age_dataset.padded_batch(
    128, (tf.TensorShape([None, 300]), tf.TensorShape([])))

iterator = padded_age_dataset.make_one_shot_iterator()
iterator.get_next()

(<tf.Tensor: id=69298, shape=(128, 175, 300), dtype=float32, numpy=
 array([[[ 0.06994925, -0.08795561,  0.10732768, ..., -0.42854732,
          -0.24935117,  0.0914242 ],
         [ 0.13329802, -0.18502815,  0.21870264, ..., -0.37015957,
          -0.14289252, -0.07735917],
         [ 0.14588733, -0.11126285, -0.14611486, ...,  0.15282993,
          -0.11073759, -0.06555343],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.11492804, -0.06971309,  0.04431322, ..., -0.2264956 ,
           0.19723506, -0.15028167],
         [-0.01100962, -0.10677161, -0.12601174, ..., -0.085996  ,
           0.1692722 ,  0.15154783],
         [-0.02378013,  0.10593007, -0.08805973, ..., -0.07255152,
           0.226

In [27]:
tf.random.set_seed(13)
lstm_age_model = tf.keras.Sequential()
lstm_age_model.add(tf.keras.layers.Masking(mask_value=0.0, input_shape=(None, 300)))
lstm_age_model.add(tf.keras.layers.LSTM(64, return_sequences=True))
lstm_age_model.add(tf.keras.layers.LSTM(32))
lstm_age_model.add(tf.keras.layers.Dense(16, activation='relu'))
lstm_age_model.add(tf.keras.layers.Dense(10, activation='relu'))

lstm_age_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      optimizer='adam', metrics=['accuracy'])
lstm_age_model.fit(x=padded_age_dataset, epochs=3, verbose=1, workers=8, use_multiprocessing=True)

Epoch 1/3
4219/4219 [==============================] - 2257s 535ms/step - loss: 1.4724 - accuracy: 0.3956
Epoch 2/3
4219/4219 [==============================] - 2224s 527ms/step - loss: 1.3742 - accuracy: 0.4302
Epoch 3/3
4219/4219 [==============================] - 2246s 532ms/step - loss: 1.3448 - accuracy: 0.4415


In [8]:
# lstm_age_model.fit(x=padded_age_dataset, epochs=2, verbose=1, workers=8, use_multiprocessing=True)

Epoch 1/2
4219/4219 [==============================] - 2528s 599ms/step - loss: 1.3864 - accuracy: 0.4286
Epoch 2/2
4219/4219 [==============================] - 2521s 597ms/step - loss: 1.3586 - accuracy: 0.4392


In [29]:
lstm_age_model.save_weights('./models/lstm_age/weights.h5')

In [30]:
with open('./models/lstm_age/json', 'w') as jsonfile:
    jsonfile.write(lstm_age_model.to_json())

In [18]:
# lstm_age_model.save('./models/lstm_age_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-06-12 06:40:40,147 : WARNING : From /opt/conda/envs/tensorflow2_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./models/lstm_age_model/assets


2020-06-12 06:40:41,225 : INFO : Assets written to: ./models/lstm_age_model/assets


In [3]:
import tensorflow as tf
with open('./models/lstm_age/json', 'r') as jsonfile:
    model = tf.keras.models.model_from_json(jsonfile.read())
model.load_weights('./models/lstm_age/weights.h5')
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      optimizer='adam', metrics=['accuracy'])

In [19]:
# import tensorflow as tf
# from tensorflow.keras.backend import manual_variable_initialization
# manual_variable_initialization(True)
# model = tf.keras.models.load_model('./models/lstm_age_model')

In [32]:
def generate_age_valid():
    for arr, age in zip(valid_x.to_numpy(), valid_y):
        yield (wv[arr], age)
        
valid_age_dataset = tf.data.Dataset.from_generator(
    generate_age_valid, (tf.float32, tf.int64),
    (tf.TensorShape([None, 300]), tf.TensorShape([])))

valid_age_dataset = valid_age_dataset.padded_batch(128, (tf.TensorShape([None, 300]),
                                                        tf.TensorShape([])))
valid_loss, valid_acc = model.evaluate(x=valid_age_dataset, workers=8,
                                                 use_multiprocessing=True)
print('valid loss', valid_loss, 'valid accuracy', valid_acc)

     78/Unknown - 21s 274ms/step - loss: 1.3523 - accuracy: 0.4408

KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
print('reading...')
predict_merged = pd.read_csv('./predict_merged.csv')
print('preprocessing...')
predict_merged['creative_id'] = predict_merged['creative_id'].astype(str)
print('transforming...')
predict_creative_id = predict_merged.groupby(by=['user_id'])['creative_id'].apply(list)
predict_creative_id

reading...
preprocessing...
transforming...


user_id
3000001    [103064, 351878, 593698, 665090, 1508864, 1711...
3000002    [12838, 12838, 37295, 37298, 37298, 37298, 475...
3000003    [20503, 36634, 41212, 50485, 68601, 73137, 115...
3000004    [9870, 63715, 63715, 63715, 94025, 108656, 157...
3000005    [17535, 26793, 40625, 49682, 71219, 123860, 16...
                                 ...                        
3999996    [12916, 30856, 54100, 65314, 116356, 167175, 1...
3999997    [14993, 50445, 66299, 271727, 390953, 641757, ...
3999998    [24188, 54150, 100456, 316582, 464055, 771243,...
3999999    [840, 9148, 71770, 72039, 78406, 95215, 115084...
4000000    [1973, 4160, 48855, 49574, 58631, 66621, 71689...
Name: creative_id, Length: 1000000, dtype: object

In [5]:
def generate_age_predict():
    for arr in predict_creative_id.to_numpy():
        yield wv[arr]
# del predict_merged
# import gc
# gc.collect()
predict_age_dataset = tf.data.Dataset.from_generator(
    generate_age_predict, tf.float32, tf.TensorShape([None, 300]))
padded_age_dataset = predict_age_dataset.padded_batch(
    32, tf.TensorShape([None, 300]))
# padded_age_dataset = predict_age_dataset.batch(1)

iterator = padded_age_dataset.make_one_shot_iterator()
iterator.get_next()

predict_age = model.predict(x=padded_age_dataset, workers=8,
                                   use_multiprocessing=True, verbose=1)
predict_age

2020-06-13 01:47:53,336 : WARNING : From <ipython-input-5-81b22c03a143>:13: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


31250/31250 [==============================] - 2647s 85ms/step


array([[ 2.1039748 ,  5.5236063 ,  8.350209  , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.1425511 ,  0.        ,  0.84054357, ...,  7.0429306 ,
         4.839012  ,  0.74673605],
       [ 4.334666  ,  6.027592  ,  5.863304  , ...,  1.4336034 ,
         0.14953418,  0.        ],
       ...,
       [ 9.732215  , 10.427769  ,  7.478134  , ...,  0.        ,
         0.        ,  0.        ],
       [ 6.1193237 , 10.445926  ,  9.378346  , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.7115227 ,  1.9410328 ,  4.360782  , ...,  3.960466  ,
         1.346945  ,  0.        ]], dtype=float32)

In [6]:
submission_age = pd.DataFrame({'user_id': predict_creative_id.index})
submission_age['predicted_age'] = np.apply_along_axis(np.argmax, 1, predict_age) + 1
submission_age['predicted_gender'] = [0 for _ in range(3000001, 4000001)]

submission_age

,user_id,predicted_age,predicted_gender
0,3000001,3,0
1,3000002,7,0
2,3000003,2,0
3,3000004,2,0
4,3000005,4,0
...,...,...,...
999995,3999996,3,0
999996,3999997,2,0
999997,3999998,2,0
999998,3999999,2,0


In [7]:
submission_age.to_csv('./submission_age.csv', index=False)